In [ ]:
#this to test if it works very well
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
def ResNet18():
    return ResNet(ResidualBlock)

In [ ]:
#Use the ResNet18 on MNIST
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
import os

#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set hyperparameter
EPOCH = 10
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01

#prepare dataset and preprocessing (MNIST -> make 3x32x32 to match network)
transform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081))
])

transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081))
])

trainset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#labels in MNIST
classes = tuple(str(i) for i in range(10))

#define ResNet18
net = ResNet18().to(device)

#define loss funtion & optimize
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

100%|██████████| 9.91M/9.91M [00:08<00:00, 1.13MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 104kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 910kB/s] 
100%|██████████| 4.54k/4.54k [00:00<00:00, 807kB/s]


In [ ]:
#train
for epoch in range(pre_epoch, EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        #prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        #forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #print ac & loss in each batch
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))

    #get the ac with testdataset in each epoch
    print('Waiting Test...')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            net.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('Test\'s ac is: %.3f%%' % (100 * correct / total))

print('Train has finished, total epoch is %d' % EPOCH)


Epoch: 1
[epoch:1, iter:1] Loss: 2.550 | Acc: 7.031% 
[epoch:1, iter:2] Loss: 2.428 | Acc: 12.891% 
[epoch:1, iter:3] Loss: 2.341 | Acc: 16.406% 
[epoch:1, iter:4] Loss: 2.279 | Acc: 18.945% 
[epoch:1, iter:5] Loss: 2.233 | Acc: 20.625% 
[epoch:1, iter:6] Loss: 2.195 | Acc: 21.354% 
[epoch:1, iter:7] Loss: 2.130 | Acc: 24.554% 
[epoch:1, iter:8] Loss: 2.065 | Acc: 27.246% 
[epoch:1, iter:9] Loss: 1.990 | Acc: 31.424% 
[epoch:1, iter:10] Loss: 1.947 | Acc: 33.438% 
[epoch:1, iter:11] Loss: 1.893 | Acc: 35.582% 
[epoch:1, iter:12] Loss: 1.841 | Acc: 37.695% 
[epoch:1, iter:13] Loss: 1.781 | Acc: 40.084% 
[epoch:1, iter:14] Loss: 1.716 | Acc: 42.522% 
[epoch:1, iter:15] Loss: 1.649 | Acc: 45.365% 
[epoch:1, iter:16] Loss: 1.599 | Acc: 47.314% 
[epoch:1, iter:17] Loss: 1.543 | Acc: 49.311% 
[epoch:1, iter:18] Loss: 1.487 | Acc: 51.476% 
[epoch:1, iter:19] Loss: 1.435 | Acc: 53.413% 
[epoch:1, iter:20] Loss: 1.389 | Acc: 55.195% 
[epoch:1, iter:21] Loss: 1.343 | Acc: 56.585% 
[epoch:1, ite